# Stable Diffusion

<https://github.com/huggingface/diffusers>

## install modules

In [ ]:
!pip install --upgrade diffusers[torch]

## import modules

In [ ]:
from diffusers import DiffusionPipeline
import torch
import time

## simple usage

initialize a pipeline

In [ ]:
pipeline = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipeline.to("cuda")

generate with prompt

In [ ]:
pipeline("An image of a squirrel in Picasso style").images[0]

In [ ]:
def __time__(func, *args, **kwargs):
    start = time.perf_counter()
    result = func(*args, **kwargs)
    end = time.perf_counter()
    elapsed = end - start
    print(f"{end - start:.6f} [s]")
    return result

In [ ]:
prompt = 'Coffee. Sunlight. A breeze through bossa nova. The world lingers here.' #@param {type:"string"}
image = __time__(pipeline, prompt).images[0]
image

In [ ]:
name = 'sunlight_cafe' #@param {type: "string"}
image.save(f'{name}.png')

## Mojo

### install mojo

In [ ]:
!pip install modular

In [ ]:
!mojo --version

### write a new file

In [ ]:
%%file import_test.mojo
import sys
from python import Python

fn main() raises:
    try:
        var diffusers = Python.import_module('diffusers')
        print(diffusers)
        print(diffusers.DiffusionPipeline)
    except:
        sys.exit(1)


In [ ]:
!mojo import_test.mojo

In [ ]:
%%file sd.mojo
import time
import sys
from python import Python

fn main() raises:
    try:
        var torch = Python.import_module('torch')
        var diffusers = Python.import_module('diffusers')
        var pipeline = diffusers.DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16)
        pipeline.to("cuda")

        var prompt = input('prompt > ')

        var start = time.monotonic()
        var image = pipeline(prompt).images[0]
        var end = time.monotonic()
        var elapsed = (end - start) / 1_000_000_000
        print(String('{} [s]').format(elapsed))

        var name = input('file name > ')
        var file_name = String('{}.png').format(name)
        image.save(file_name)
        print(String('{} saved').format(file_name))
    except:
        sys.exit(1)


In [ ]:
!mojo build sd.mojo

### run

In [ ]:
!./sd

## Python vs Mojo

Prompt: `Coffee. Sunlight. A breeze through bossa nova. The world lingers here.`

||Python|Mojo|
|:-:|-:|-:|
|1|8.987383|8.749698725|
|2|9.248939|9.048349062|
|3|7.852054|8.657060594|
|4|7.934670|8.686166867|
|5|7.947472|9.687662112|
|6|7.984817|8.594926555|
|7|7.825276|9.673236295|
|8|7.981634|9.194612269|
|9|7.965697|9.547111223|
|10|7.819419|9.426520401|

In [ ]:
markdown_table = """
||Python|Mojo|
|:-:|-:|-:|
|1|8.987383|8.749698725|
|2|9.248939|9.048349062|
|3|7.852054|8.657060594|
|4|7.934670|8.686166867|
|5|7.947472|9.687662112|
|6|7.984817|8.594926555|
|7|7.825276|9.673236295|
|8|7.981634|9.194612269|
|9|7.965697|9.547111223|
|10|7.819419|9.426520401|
"""

In [ ]:
import pandas as pd
import io

In [ ]:
df = pd.read_csv(io.StringIO(markdown_table), sep='|', skiprows=[0, 1]).dropna(axis=1, how='all').iloc[0:]
df.columns = ['Index', 'Python', 'Mojo']
df = df.set_index('Index')
df = df.apply(pd.to_numeric, errors='coerce')

df

In [59]:
average_python = df['Python'].mean()
average_mojo = df['Mojo'].mean()

print(f"Average Python execution time: {average_python:.6f} [s]")
print(f"Average Mojo execution time: {average_mojo:.6f} [s]")

Average Python execution time: 8.154736 [s]
Average Mojo execution time: 9.126534 [s]
